# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846760218771                   -0.70    4.5         
  2   -7.852314705153       -2.26       -1.53    1.0   29.0ms
  3   -7.852616182827       -3.52       -2.55    1.5   32.4ms
  4   -7.852645999704       -4.53       -2.89    2.5   39.9ms
  5   -7.852646516508       -6.29       -3.19    1.0   29.9ms
  6   -7.852646679904       -6.79       -4.08    1.0   30.7ms
  7   -7.852646686418       -8.19       -5.18    1.8   36.3ms
  8   -7.852646686724       -9.51       -5.52    1.8   38.0ms
  9   -7.852646686730      -11.28       -6.25    1.0   30.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846938643063                   -0.70           4.8         
  2   -7.852552571681       -2.25       -1.64   0.80    2.0    323ms
  3   -7.852637916108       -4.07       -2.76   0.80    1.0   95.5ms
  4   -7.852646459256       -5.07       -3.29   0.80    1.8   32.2ms
  5   -7.852646663625       -6.69       -4.13   0.80    1.0   27.0ms
  6   -7.852646686404       -7.64       -4.72   0.80    2.2   37.2ms
  7   -7.852646686723       -9.50       -5.50   0.80    1.5   30.3ms
  8   -7.852646686729      -11.21       -6.83   0.80    1.8   32.5ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.397087e+01     4.083326e+00
 * time: 0.0709841251373291
     1     1.488623e+00     2.244917e+00
 * time: 0.3407731056213379
     2    -1.393294e+00     2.664176e+00
 * time: 0.3707890510559082
     3    -3.899744e+00     2.423689e+00
 * time: 0.4134190082550049
     4    -5.358031e+00     2.220588e+00
 * time: 0.455808162689209
     5    -6.932213e+00     1.142576e+00
 * time: 0.4988851547241211
     6    -7.124413e+00     1.308915e+00
 * time: 0.5287120342254639
     7    -7.578899e+00     7.803574e-01
 * time: 0.5582411289215088
     8    -7.677829e+00     1.023438e+00
 * time: 0.58833909034729
     9    -7.750469e+00     7.625851e-01
 * time: 0.6177921295166016
    10    -7.780377e+00     9.816255e-01
 * time: 0.6469810009002686
    11    -7.804420e+00     7.158544e-01
 * time: 0.6763291358947754
    12    -7.828035e+00     2.858986e-01
 * time: 0.7183871269226074
    13    -7.843172e+00     1.842606e-01
 * time: 0.7471570968627

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846783662574                   -0.70    4.2         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645865933                   -1.64         
  2   -7.852646686730       -6.09       -3.71    2.46s
  3   -7.852646686730      -13.18       -7.21    207ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.013500126227422e-7
|ρ_newton - ρ_scfv| = 1.6446929409080616e-7
|ρ_newton - ρ_dm|   = 2.04461250253369e-9
